In [ ]:
!pip install -U skorch

In [ ]:
import pandas as pd
import numpy as np
import torch.nn.functional as F
import matplotlib.pyplot as plt
from pandas_profiling import ProfileReport
from sklearn.experimental import enable_hist_gradient_boosting
from skorch import NeuralNetRegressor
from torch import nn
import torch.optim as optim
import torch.nn.functional as F
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold, StratifiedKFold, GroupKFold
from tqdm import tqdm_notebook as tqdm
import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error
import gc
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/train.csv")
test = pd.read_csv("/kaggle/input/tabular-playground-series-jan-2021/test.csv")
sample_submission = pd.read_csv('../input/tabular-playground-series-jan-2021/sample_submission.csv')

In [ ]:
target = train['target'].values.reshape(-1, 1)
columns = test.columns[1:]

In [ ]:
train_oof = np.zeros((300000,))
test_preds = 0
train_oof.shape

In [ ]:
train

In [ ]:
target

In [ ]:
class RegressorModule(nn.Module):
    def __init__(
            self,
            num_units=100,
            nonlin=F.relu,
    ):
        super(RegressorModule, self).__init__()
        self.num_units = num_units
        self.nonlin = nonlin

        self.dense0 = nn.Linear(14, num_units)
        self.nonlin = nonlin
        self.dense1 = nn.Linear(num_units, 10)
        self.output = nn.Linear(10, 1)

    def forward(self, X, **kwargs):
        X = self.nonlin(self.dense0(X))
        X = F.relu(self.dense1(X))
        X = self.output(X)
        return X

In [ ]:
NUM_EPOCHS = 25 
NUM_FOLDS = 10
kf = KFold(n_splits=NUM_FOLDS, shuffle=True, random_state=0)

for f, (train_ind, val_ind) in tqdm(enumerate(kf.split(train, target))):
        #print(f'Fold {f}')
        train_df, val_df = train.iloc[train_ind][columns].values.astype(np.float32), train.iloc[val_ind][columns].values.astype(np.float32)
        train_target, val_target = target[train_ind].astype(np.float32), target[val_ind].astype(np.float32)
        
        model =  NeuralNetRegressor(RegressorModule, lr=0.01, criterion=nn.MSELoss, max_epochs=NUM_EPOCHS, optimizer=optim.Adam, device='cuda')
        model.fit(train_df, train_target)
        temp_oof = model.predict(val_df)
        temp_test = model.predict(test[columns].values.astype(np.float32))

        train_oof[val_ind] = temp_oof.flatten()
        test_preds += temp_test.flatten()/NUM_FOLDS
        print("Fold error:", mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
print("Fold error:", mean_squared_error(temp_oof, val_target, squared=False))

In [ ]:
np.save('train_oof', train_oof)
np.save('test_preds', test_preds)

In [ ]:
sample_submission['target'] = test_preds
sample_submission.to_csv('submission.csv', index=False)